In [30]:
import pandas as pd
from pandas import ExcelWriter
from pandas import ExcelFile

df = pd.read_excel('../../Produktion/TrainingData/TrainingData_5cat_de.xlsx')
df2 = pd.read_excel('../backend/model_data/TrainingData_5cat_de.xlsx')


print("Column headings:")
print(df.columns)

Column headings:
Index(['Unnamed: 0', 'platform', 'User', 'topic', 'phase/arg_situation',
       'method', 'adressee, reference', 'function', 'intent', 'impl_expl',
       'pos_neg', 'easy translation', 'link', 'connected', 'utterance',
       'Effekt', 'lecture', 'praise', 'insinuation', 'dissence', 'concession'],
      dtype='object')


In [31]:
from collections import Counter, OrderedDict

category_names = Counter()
for i in df.index:
    category_names[df['Effekt'][i]] +=1

for entry in category_names.most_common():
    print(entry)


('lecture', 245)
('insinuation', 194)
('dissence', 163)
('concession', 98)
('praise', 86)
('revelation', 39)
(nan, 35)
('discreditation', 15)
('support', 14)
('sarcasm', 14)
('demand', 10)
('solution', 9)
('speculation', 8)
('colonisation', 6)
('challenge', 6)
('claim', 6)
('accusation', 6)
('indication', 6)
('conspiracy', 4)
('judgement', 3)
('humour', 3)
('relativation', 2)
('warning', 2)
('ambivalence', 2)
('consent', 2)
('attack', 2)
('irony', 2)
('question', 2)
('reflection', 1)
('gratitude', 1)


In [204]:
import re

text = "Das sind (keine|zwei|eine) Enteignungen, .* nur die (Gerechtigkeit|Vollzug|Urinente|Grundrente), das ist Volksherrschaft (digitaletechnik)"
patterns = [ 
    r'\.\*', 
    r'\([\w+\|\w+]+\)', 
    r'\([\w+(?!\))]+\)',
    r'\([\w+(?<=\|)\w+]+\)',
    
]
for pattern in patterns:
    regex = re.compile(pattern)
    print('matched: ', regex.findall(text))

for utt in df['utterance']:
    for pattern in patterns:
        regex = re.compile(pattern)
        if regex.search(utt):
            print(utt)


matched:  ['.*']
matched:  ['(keine|zwei|eine)', '(Gerechtigkeit|Vollzug|Urinente|Grundrente)', '(digitaletechnik)']
matched:  ['(digitaletechnik)']
matched:  ['(keine|zwei|eine)', '(Gerechtigkeit|Vollzug|Urinente|Grundrente)', '(digitaletechnik)']
.* (Seien|Sei)  .* nicht .*  albern .*
.* (Seien|Sei)  .* nicht .*  albern .*
.* (Seien|Sei)  .* nicht .*  albern .*
Das ist doch albern .*
.* (Seien|Sei)  .* nicht .*  albern .*
.* (Seien|Sei)  .* nicht .*  albern .*
.* (Seien|Sei)  .* nicht .*  albern .*
das ist das Lächerlichste, .* (das|was) .* ich je gehört habe
das ist das Lächerlichste, .* (das|was) .* ich je gehört habe
das ist das Lächerlichste, .* (das|was) .* ich je gehört habe
.* Lächerlichste, (das|was|von|wovon) .* je gehört .*
.* Lächerlichste, (das|was|von|wovon) .* je gehört .*
.* Lächerlichste, (das|was|von|wovon) .* je gehört .*
.* (Das|sowas|so etwas) .* ist .* (sehr|doch) .* (albern|lächerlich|lachhaft) .*
.* (Das|sowas|so etwas) .* ist .* (sehr|doch) .* (albern|lächerli